In [1]:
import os
import pandas as pd
from datetime import datetime

In [2]:
def mascara(df, coluna,n_digitos):
    df[coluna] = df[coluna].astype(str)
    df[coluna] = str("000") + df[coluna]
    df[coluna] = df[coluna].astype(str).str[-n_digitos:] 
    return df

In [3]:
path_retalhonamento = r'\\GLWFS02.lwart.net\LWC-FLORESTAL\Planejamento_e_Controle\Geoprocessamento\02_SETOR\07_CADASTRO\14_BASE RETALHONAMENTOS\DExPARA\SHP'

In [4]:
arquivos_retalhonamentos = []



# Caminhar pela árvore de diretórios
for root, dirs, files in os.walk(path_retalhonamento):
    for a in files:
        if a.endswith('.xlsx') and 'de_para' in a.lower() and '~' not in a and not 'parcial' in a.lower() :
            file_path = os.path.join(root, a)
            creation_time = os.path.getctime(file_path)
            creation_date = datetime.fromtimestamp(creation_time).strftime('%Y-%m-%d')
            origem = os.path.basename(root)
            nome_do_arquivo = a
            arquivos_retalhonamentos.append((file_path, a, creation_date, origem))
            


colunas_ret = ['Arquivo', 'nome do arquivo', 'data','diretorio']
links_retalhonamento = pd.DataFrame(arquivos_retalhonamentos, columns=colunas_ret)
links_retalhonamento = links_retalhonamento.sort_values(['data'], ascending=False).drop_duplicates(['Arquivo'], keep='first')

In [5]:
links_retalhonamento

,Arquivo,nome do arquivo,data,diretorio
14,\\GLWFS02.lwart.net\LWC-FLORESTAL\Planejamento...,2024_06_DE_PARA - FAZENDAS SÃO PAULO - JUNHO 2...,2024-06-27,20240627
13,\\GLWFS02.lwart.net\LWC-FLORESTAL\Planejamento...,2024_06_DE_PARA - FAZENDAS SÃO PAULO - JUNHO 2...,2024-06-20,20240620
12,\\GLWFS02.lwart.net\LWC-FLORESTAL\Planejamento...,2024_06_DE_PARA - FAZENDAS SÃO PAULO - JUNHO 2...,2024-06-11,20240611
11,\\GLWFS02.lwart.net\LWC-FLORESTAL\Planejamento...,2024_06_DE_PARA - FAZENDAS SÃO PAULO - JUNHO 2...,2024-06-03,20240603
10,\\GLWFS02.lwart.net\LWC-FLORESTAL\Planejamento...,2024_05_DE_PARA - FAZENDAS SÃO PAULO - MAIO 20...,2024-05-27,20240527
9,\\GLWFS02.lwart.net\LWC-FLORESTAL\Planejamento...,2024_05_DE_PARA - FAZENDAS SÃO PAULO - MAIO 20...,2024-05-17,20240517
8,\\GLWFS02.lwart.net\LWC-FLORESTAL\Planejamento...,2024_05_DE_PARA - FAZENDAS SÃO PAULO - MAIO 20...,2024-05-10,20240510
7,\\GLWFS02.lwart.net\LWC-FLORESTAL\Planejamento...,2024_04_DE_PARA - FAZENDAS SÃO PAULO - ABRIL 2...,2024-05-02,20240502
6,\\GLWFS02.lwart.net\LWC-FLORESTAL\Planejamento...,2024_03_DE_PARA - FAZENDAS SÃO PAULO - MARÇO 2...,2024-04-01,20240401
5,\\GLWFS02.lwart.net\LWC-FLORESTAL\Planejamento...,2024_02_DE_PARA - FAZENDAS SÃO PAULO - FEVEREI...,2024-03-01,20240301


In [6]:
retalhonamentos = pd.DataFrame()
for _, n in links_retalhonamento.iterrows():
    base = pd.read_excel(n['Arquivo'],skiprows=1)
    base['nome_do_arquivo'] = n['nome do arquivo']
    retalhonamentos = pd.concat([retalhonamentos,base], ignore_index=True)

In [7]:
#Escolher colunas para importar base de retalhonamento


retalhonamentos = retalhonamentos[['CD_USO_SOL','ID_PROJETO', 'ID_TALHAO', 'DATA_CRIAC', 'CD_USO_SOL_1','ID_PROJETO_1', 'ID_TALHAO_1', 'DATA_CRIAC_1']]


#Criando coluna de apoio para retalhonamentos

#Definindo o formato de dados dentro a base de retalhonamentos
retalhonamentos = mascara(retalhonamentos,'ID_PROJETO',4)
retalhonamentos = mascara(retalhonamentos,'ID_TALHAO',3)
retalhonamentos['objeto de locação'] = retalhonamentos['ID_PROJETO'] + retalhonamentos['ID_TALHAO']
retalhonamentos = mascara(retalhonamentos,'objeto de locação',7)

#Preenchendo campos vazios com zero para transformar em str
retalhonamentos['ID_PROJETO_1'] = retalhonamentos['ID_PROJETO_1'].fillna(0).astype(int).astype(str)
retalhonamentos['ID_TALHAO_1'] = retalhonamentos['ID_TALHAO_1'].fillna(0).astype(int).astype(str)

#Definindo o formato de dados dentro a base de retalhonamentos
retalhonamentos = mascara(retalhonamentos,'ID_PROJETO_1',4)
retalhonamentos = mascara(retalhonamentos,'ID_TALHAO_1',3)
retalhonamentos['objeto de locação_1'] = retalhonamentos['ID_PROJETO_1'] + retalhonamentos['ID_TALHAO_1']
retalhonamentos = mascara(retalhonamentos,'objeto de locação_1',7)


#Definindo o tipo de dado
dtype_retalhonamento = {'DATA_CRIAC':'datetime64[ns]'}
retalhonamentos = retalhonamentos.astype(dtype_retalhonamento)


#Filtrando o que não tem cd_uso_solo definido
retalhonamentos = retalhonamentos[pd.notna(retalhonamentos['CD_USO_SOL'])]

#Encontrando o que temos de retalhonamento

#Precisamos que os talhões sejam diferentes tanto quanto o cd_uso_solo
retalhonamentos['delta_talhao'] = (retalhonamentos['CD_USO_SOL'] == retalhonamentos['CD_USO_SOL_1']) | (retalhonamentos['objeto de locação'] == retalhonamentos['objeto de locação_1'])

#Filtrando somente talhões que tem identificação de retalhonameto
retalhonamentos = retalhonamentos[retalhonamentos['delta_talhao'] == False]


#Organizar talhões de forma ascendente

retalhonamentos = retalhonamentos.sort_values(['CD_USO_SOL','CD_USO_SOL_1'], ascending=[False,False])


In [8]:
retalhonamentos

,CD_USO_SOL,ID_PROJETO,ID_TALHAO,DATA_CRIAC,CD_USO_SOL_1,ID_PROJETO_1,ID_TALHAO_1,DATA_CRIAC_1,objeto de locação,objeto de locação_1,delta_talhao
11239,104503.0,0692,041,2024-06-18 00:00:00.000000000,98830.0,0692,021,2024-06-24,0692041,0692021,False
42927,104058.0,2095,301,1970-01-01 00:00:00.000045433,98832.0,2095,001,2024-06-07,2095301,2095001,False
42936,104057.0,2095,315,1970-01-01 00:00:00.000045433,98839.0,2095,015,2024-06-07,2095315,2095015,False
42928,104056.0,2095,302,1970-01-01 00:00:00.000045433,98833.0,2095,002,2024-06-07,2095302,2095002,False
42937,104055.0,2095,316,1970-01-01 00:00:00.000045433,98840.0,2095,016,2024-06-07,2095316,2095016,False
...,...,...,...,...,...,...,...,...,...,...,...
37594,10630.0,0480,014,1970-01-01 00:00:00.000045225,13598.0,0480,015,2024-06-07,0480014,0480015,False
111953,10204.0,0014,002,2024-02-09 00:00:00.000000000,102421.0,0014,068,2024-03-20,0014002,0014068,False
148690,10129.0,0327,005,2020-11-21 00:00:00.000000000,99449.0,0327,024,2024-01-26,0327005,0327024,False
148574,10089.0,0322,007,2020-08-19 00:00:00.000000000,99446.0,0322,014,2024-01-26,0322007,0322014,False


In [9]:
retalhonamentos.to_excel('base_retalhonamentos.xlsx')